In [58]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [59]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [60]:
import warnings 
warnings.filterwarnings("ignore")

In [61]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [62]:
# train_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/train.csv")
# test_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")
# train_data.shape, test_data.shape

In [63]:
# train_data.head()

In [64]:
# train_data = train_data.drop("id", axis = 1)

In [65]:
# train_data.corr()["FloodProbability"]

In [66]:
# X = train_data.drop("FloodProbability", axis = 1)
# y = train_data["FloodProbability"]

# test df

In [67]:
# test_data.drop("id", axis = 1, inplace = True)

In [68]:
# column_names = X.columns

In [69]:
# sc = StandardScaler()
# X = sc.fit_transform(X)
# test_data = sc.transform(test_data)

In [70]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

## Model training
documentation : https://www.kaggle.com/code/carlmcbrideellis/gpu-accelerated-shap-values-jane-street-example

In [71]:
# model_params = {
#     "n_estimators" : 1000,
#     "learning_rate" : 0.01,
#     "max_depth" : 10,
#     "device" : device
# }

In [72]:
# eval_set = [(X_val, y_val)]

In [73]:
# model_XGB = XGBRegressor(**model_params)
# model_XGB.fit(X_train, y_train, eval_set=eval_set, verbose = False)

## Experiments

In [74]:
# temp = train_data.copy()
# column_names

In [75]:
# temp.head()

In [76]:
# X = temp.drop("FloodProbability", axis = 1)
# y = temp["FloodProbability"]

In [77]:
# for i, column in enumerate(column_names):
#     if i == 0:
#         continue
#     X[f"sum_of_{i}"] = X[list(column_names[:i])].sum(axis = 1)

In [78]:
# X.head()

In [79]:
# test_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")
# test_data.drop("id", axis = 1, inplace = True)
# for i, column in enumerate(column_names):
#     if i == 0:
#         continue
#     test_data[f"sum_of_{i}"] = test_data[list(column_names[:i])].sum(axis = 1)

In [80]:
# X = sc.fit_transform(X)
# test_data = sc.transform(test_data)

In [81]:
# model_XGB.fit(X, y, verbose = False)

## ANN

In [82]:
train_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/train.csv")
test_data = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")

train_data = train_data.drop("id", axis = 1)
test_data = test_data.drop("id", axis = 1)

X = train_data.drop("FloodProbability", axis = 1)
y = train_data["FloodProbability"]

X["sum_all_val"] = X.sum(axis = 1)
test_data["sum_all_val"] = test_data.sum(axis = 1)

sc = StandardScaler()
X = sc.fit_transform(X)
test_data = sc.transform(test_data)

In [83]:
import torch
from torch import nn
from torch.nn import functional as F

class RegressionNet(nn.Module):
    def __init__(self, n_features):
        super(RegressionNet, self).__init__()
        self.hidden1 = nn.Linear(n_features, 2048)
        self.hidden2 = nn.Linear(2048, 1024)
        self.hidden3 = nn.Linear(1024, 512)
        self.hidden4 = nn.Linear(512, 264)
        self.hidden5 = nn.Linear(264, 128)
        self.hidden6 = nn.Linear(128, 64)
        self.hidden7 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
    
    def forward(self, x):
      # Pass through the first hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden1(x))
      x = F.dropout(x, p=0.2)

      # Pass through the second hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden2(x))
      x = F.dropout(x, p=0.2)

      # Pass through the third hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden3(x))
      x = F.dropout(x, p=0.2)

      # Pass through the fourth hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden4(x))
      x = F.dropout(x, p=0.2)

      # Pass through the fifth hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden5(x))
      x = F.dropout(x, p=0.2)

      # Pass through the sixth hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden6(x))
      x = F.dropout(x, p=0.2)

      # Pass through the seventh hidden layer with ReLU activation and dropout
      x = F.relu(self.hidden7(x))
      x = F.dropout(x, p=0.2)

      # Final output layer without activation (linear regression)
      return self.output(x)

In [84]:
n_features = X.shape[1]
n_features

21

In [85]:
X = torch.from_numpy(X.astype(np.float32))
y = torch.tensor(y.values.astype(np.float32))
test_data = torch.from_numpy(test_data.astype(np.float32))

In [86]:
X.dtype

torch.float32

In [87]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]
        return x, y

In [88]:
dataset = CustomDataset(X, y)

In [89]:
batch_size = 5120
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [90]:
from tqdm.auto import tqdm

model = RegressionNet(n_features)
model.to(device)

loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters())

loss_vals = []

# pb = tqdm(total = 50, desc = "Training")

model.train()

for epoch in tqdm(range(30)):
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)

        # Forward pass
        y_pred = model(data)
        loss = loss_fn(y_pred.squeeze(), target)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(data_loader)
    loss_vals.append(avg_loss)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")
#     pb.set_postfix(loss=avg_loss)
#     pb.update(1)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Loss: 0.0065
Epoch: 2, Loss: 0.0020
Epoch: 3, Loss: 0.0014
Epoch: 4, Loss: 0.0012
Epoch: 5, Loss: 0.0009
Epoch: 6, Loss: 0.0008
Epoch: 7, Loss: 0.0006
Epoch: 8, Loss: 0.0006
Epoch: 9, Loss: 0.0005
Epoch: 10, Loss: 0.0005
Epoch: 11, Loss: 0.0005
Epoch: 12, Loss: 0.0005
Epoch: 13, Loss: 0.0005
Epoch: 14, Loss: 0.0005
Epoch: 15, Loss: 0.0005
Epoch: 16, Loss: 0.0005
Epoch: 17, Loss: 0.0005
Epoch: 18, Loss: 0.0005
Epoch: 19, Loss: 0.0005
Epoch: 20, Loss: 0.0005
Epoch: 21, Loss: 0.0005
Epoch: 22, Loss: 0.0005
Epoch: 23, Loss: 0.0005
Epoch: 24, Loss: 0.0005
Epoch: 25, Loss: 0.0005
Epoch: 26, Loss: 0.0005
Epoch: 27, Loss: 0.0005
Epoch: 28, Loss: 0.0005
Epoch: 29, Loss: 0.0005
Epoch: 30, Loss: 0.0005


In [91]:
torch.save(model.state_dict(), f"ann_flood.pth")

# Submission

In [92]:
model = RegressionNet(n_features)
model.load_state_dict(torch.load("/kaggle/working/ann_flood.pth"))
model.to(device)
model.eval()

RegressionNet(
  (hidden1): Linear(in_features=21, out_features=2048, bias=True)
  (hidden2): Linear(in_features=2048, out_features=1024, bias=True)
  (hidden3): Linear(in_features=1024, out_features=512, bias=True)
  (hidden4): Linear(in_features=512, out_features=264, bias=True)
  (hidden5): Linear(in_features=264, out_features=128, bias=True)
  (hidden6): Linear(in_features=128, out_features=64, bias=True)
  (hidden7): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
)

In [93]:
class CustomDataset(Dataset):
    def __init__(self, features):
        self.features = features
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        return x


test_dataset = CustomDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=4096, shuffle=False, num_workers=2)

all_predictions = []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        predictions = model(data)
        all_predictions.append(predictions.cpu().numpy())  # to CPU and convert to numpy array

In [94]:
all_predictions = np.concatenate(all_predictions)
all_predictions

array([[0.5606198 ],
       [0.4630538 ],
       [0.44375226],
       ...,
       [0.63070434],
       [0.55879116],
       [0.5289618 ]], dtype=float32)

In [95]:
test_data_id = pd.read_csv(r"/kaggle/input/playground-series-s4e5/test.csv")["id"]

In [96]:
submission = pd.DataFrame(list(zip(test_data_id, all_predictions)), columns = ["id", "FloodProbability"])
submission.head()

,id,FloodProbability
0,1117957,[0.5606198]
1,1117958,[0.4630538]
2,1117959,[0.44375226]
3,1117960,[0.46852297]
4,1117961,[0.46923712]


In [97]:
submission["FloodProbability"] = submission["FloodProbability"].apply(lambda x : x[0])
submission.head()

,id,FloodProbability
0,1117957,0.560620
1,1117958,0.463054
2,1117959,0.443752
3,1117960,0.468523
4,1117961,0.469237


In [98]:
submission.to_csv("submission.csv", index = False)

# SHAP

In [99]:
# import xgboost as xgb
# import shap
# shap.initjs()

In [100]:
# dmat = xgb.DMatrix(X, y)
# bst  = xgb.train({"tree_method": "gpu_hist"}, dmat, 500)
# # Make sure GPU prediction is enabled
# bst.set_param({"predictor": "gpu_predictor"})

In [101]:
# %time
# shap_values = bst.predict(dmat, pred_contribs=True)

In [102]:
# shap.summary_plot(shap_values[:,:-1], X)